# **Lexicon**

In [1]:
import numpy as np
import pandas as pd

### Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### DB 연결
# !pip install pymysql
import pymysql

### DB에 저장
import sqlalchemy
from sqlalchemy import create_engine

### 실시간 주식가격 데이터
# !pip install finance-datareader
import FinanceDataReader as fdr

### 텍스트 분석
## KoNLPy
# 1) JAVA 설치, 2) Python 버전과 맞는 JPype1-py3 설치, 3) !pip install konlpy, 4) 설치 경로에서 jvm.py 파일 코드 67번 줄 주석 처리 
from konlpy.tag import Okt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
## FastText
# !pip install gensim
# !pip install fasttext
import fasttext
import fasttext.util
# Facebook 한국어 Embedding 모델 다운로드 → 한 번만 설치하면 됨
fasttext.util.download_model('ko', if_exists='ignore')
# 유사도 계산
from gensim import models

### 모델 학습 및 평가
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score

### 모델 저장 및 로드
import joblib

### 기타
import datetime
from collections import Counter
import sys
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf

print('GPU', '사용 가능' if tf.config.experimental.list_physical_devices('GPU') else '사용 불가능')

GPU 사용 불가능


In [3]:
!nvidia-smi

Sun Sep 19 08:30:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 457.49       Driver Version: 457.49       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 1650   WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   43C    P8     1W /  N/A |    420MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## **Read KOSELF & Test Data**

### **① KOSELF 감성 어휘 사전**

In [4]:
# 블로그에서 가져온 기본적인 한국어 긍부정 텍스트 목록
with open('positive_words_self.txt', encoding='utf-8') as pos_blog:
    positive_blog = pos_blog.readlines()
positive_blog = [pos_blog.replace('\n', '') for pos_blog in positive_blog]
with open('negative_words_self.txt', encoding='utf-8') as neg_blog:
    negative_blog = neg_blog.readlines()
negative_blog = [neg_blog.replace('\n', '') for neg_blog in negative_blog]

# KOSELF 감성 어휘 사전
with open('KOSELF_pos.txt', encoding='utf-8') as pos:
    positive = pos.readlines()
positive = [pos.replace('\n', '') for pos in positive]
with open('KOSELF_neg.txt', encoding='utf-8') as neg:
    negative = neg.readlines()
negative = [neg.replace('\n', '') for neg in negative]

### **② 2020년 SK Data**

In [5]:
lexicon_2020 = pd.read_csv('../../../../Code/Data/Test/Stock-Year/2020_sk.csv')

lexicon_2020.head(1)

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization
0,sk하이닉스,660,매일경제,2020010104,美中리스크 완화에 D램 수요회복 `기대감`,http://news.mk.co.kr/newsRead.php?no=331&year=...,◆ 2020 경제기상도 / 업종별 전망 (반도체) ◆\r\n\n\n삼성전자는 신년에...,2020-01-02,4,96000,96200,94100,94700,2342070,0.006376,1,0,경제 기상도 업종 전망 반도체 전자 신년 낸드 플래시 메모리 반도체 세계 각오 사진...


In [6]:
lexicon_2020[lexicon_2020['text'].isna()]

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization


In [7]:
lexicon_2020.dropna(axis=0, inplace=True)
lexicon_2020 = lexicon_2020.reset_index(drop=True)
lexicon_2020.isna().sum()

st_n                 0
st_cd                0
news                 0
datetime             0
title                0
url                  0
text                 0
date                 0
time                 0
Open                 0
High                 0
Low                  0
Close                0
Volume               0
Change               0
UpDown               0
Extremely_Changed    0
Tokenization         0
dtype: int64

## **Calculate Cosine Similarity**

In [8]:
# 이미 학습된 FastText 내장 한국어 모델
try:
    print(ko_model, '모델 로드 불필요')
except:
    ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin')

In [9]:
limit_number = 15000
sys.setrecursionlimit(limit_number)

In [10]:
pos_lexicon_2020 = pd.DataFrame(columns=['date', 'news_num', 'KOSELF_pos_word', 'news_word', 'cosine_similarity', 'frequency'])   # 'frequency' 구현할 방법은 추후 생각...ㅠㅠ

# KOSELF_pos와의 Cosine Similarity 계산
for x in range(len(lexicon_2020['Tokenization'])):
    news_num = x+1
    for y in range(len(positive)):
        for z in range(len(list(set(lexicon_2020['Tokenization'][x].split())))):
            if (ko_model.wv.similarity(positive[y], list(set(lexicon_2020['Tokenization'][x].split()))[z]) >= 0.5) and (ko_model.wv.similarity(positive[y], list(set(lexicon_2020['Tokenization'][x].split()))[z]) != 1.0):
                freq = 0
                for w in range(len(lexicon_2020['Tokenization'][x].split())):
                    if lexicon_2020['Tokenization'][x].split()[w] == list(set(lexicon_2020['Tokenization'][x].split()))[z]:
                        freq += 1
                data = {
                    'date': lexicon_2020['date'][x],
                    'news_num': news_num,
                    'KOSELF_pos_word': positive[y],
                    'news_word': list(set(lexicon_2020['Tokenization'][x].split()))[z],
                    'cosine_similarity': ko_model.wv.similarity(positive[y], list(set(lexicon_2020['Tokenization'][x].split()))[z]),
                    'frequency': freq
                    }
                pos_lexicon_2020 = pos_lexicon_2020.append(data, ignore_index=True)
                print("***{0} Cosine Similarity between <{1}> & <{2}> : {3}".format(lexicon_2020['date'][x], positive[y], list(set(lexicon_2020['Tokenization'][x].split()))[z], ko_model.wv.similarity(positive[y], list(set(lexicon_2020['Tokenization'][x].split()))[z])))

***2020-01-02 Cosine Similarity between <본격적인> & <본격> : 0.6104379296302795
***2020-01-02 Cosine Similarity between <중요한> & <가장> : 0.5371253490447998
***2020-01-02 Cosine Similarity between <증대> & <확대> : 0.5193238258361816
***2020-01-02 Cosine Similarity between <증대> & <급증> : 0.5051040649414062
***2020-01-02 Cosine Similarity between <증대> & <감소> : 0.6517642736434937
***2020-01-02 Cosine Similarity between <호황> & <업황> : 0.5544542670249939
***2020-01-02 Cosine Similarity between <강세> & <급락> : 0.507910966873169
***2020-01-02 Cosine Similarity between <개선> & <향상> : 0.5878475308418274
***2020-01-02 Cosine Similarity between <개선> & <개선> : 0.9999999403953552
***2020-01-02 Cosine Similarity between <경신> & <기록> : 0.522990882396698
***2020-01-02 Cosine Similarity between <증대> & <증가> : 0.6946426630020142
***2020-01-02 Cosine Similarity between <증대> & <확대> : 0.5193238258361816
***2020-01-02 Cosine Similarity between <증대> & <급증> : 0.5051040649414062
***2020-01-02 Cosine Similarity between <증대> & <향상

In [11]:
pos_lexicon_2020

,date,news_num,KOSELF_pos_word,news_word,cosine_similarity,frequency
0,2020-01-02,1,본격적인,본격,0.610438,3
1,2020-01-02,1,중요한,가장,0.537125,1
2,2020-01-02,1,증대,확대,0.519324,1
3,2020-01-02,1,증대,급증,0.505104,1
4,2020-01-02,1,증대,감소,0.651764,1
...,...,...,...,...,...,...
13429,2021-01-04,2399,증대,확대,0.519324,1
13430,2021-01-04,2399,증대,급증,0.505104,1
13431,2021-01-04,2399,증대,감소,0.651764,1
13432,2021-01-04,2399,추진,사업,0.521374,3


In [12]:
pos_lexicon_2020.to_csv('../../../../Code/Data/Test/Stock-Year/pos_sk_2020.csv', index=False)

In [13]:
a = list(set(list(pos_lexicon_2020['news_word'])))

a

['황규환',
 '임병민',
 '중인',
 '진행',
 '정용배',
 '신욱수',
 '최선',
 '산곡역',
 '계획',
 '펜데믹',
 '성근로자',
 '활기',
 '남완호',
 '공유진',
 '샘코샘코',
 '확보',
 '약세',
 '차정훈',
 '대림오토바이',
 '두부과자',
 '논캡티브',
 '텍오스코텍',
 '증대',
 '저하',
 '펀인터렉티브',
 '씽크빅웅진씽크빅',
 '토픽스지수',
 '견조했',
 '나정환',
 '정찬',
 '스제약위더스',
 '훈풍',
 '적극',
 '스피폭스',
 '증가',
 '자공시',
 '문제해결',
 '이천도호부',
 '견조할',
 '김현조',
 '렛세콰이어',
 '방안',
 '텍드림텍',
 '공영은',
 '증진',
 '전철수',
 '확대',
 '급락',
 '벽체',
 '보통신',
 '언택트',
 '스지누스',
 '밑돈',
 '경제원',
 '엠에스',
 '구현',
 '쎄미켐동진쎄미켐',
 '깡통계좌',
 '재택명령',
 '탠텀전',
 '수츠윈',
 '강점',
 '박도수',
 '웃돌',
 '의미',
 '언택트주',
 '윤자민',
 '신해용',
 '충족',
 '이주수',
 '전개',
 '촉진',
 '켁스트',
 '요구',
 '게임컨솔향',
 '강력',
 '아달리무맙',
 '맹열하',
 '확충',
 '외부펀딩',
 '황선관',
 '폭락',
 '호캉스족',
 '횡보세',
 '재원임',
 '발전',
 '장점',
 '장단점',
 '랙티브',
 '이홍덕',
 '쇼와덴코',
 '자이앤트',
 '화훼이',
 '폐쇄하거',
 '확립',
 '문제',
 '조정시기',
 '해법',
 '시너지',
 '수펙스홀',
 '방법',
 '기록',
 '센티먼트',
 '오벤터스',
 '리레이팅',
 '맥쿼리인프라',
 '젬백스젬백스',
 '업황',
 '급증',
 '최고급',
 '메디앙스',
 '과학비즈니스벨트',
 '불황',
 '프로그램매매',
 '중요',
 '임홍택',
 '낙숫물',
 '스맨트',
 '량젠쟝',
 '고취',
 '취합하',
 '향상',


In [14]:
b = list(set(list(pos_lexicon_2020[pos_lexicon_2020['cosine_similarity']>=0.7]['news_word'])))

b

['쎄미켐동진쎄미켐', '추진', '경의', '실패', '단점', '개선', '최선', '약세', '불황', '장점', '충족', '밑돈']

In [15]:
c = list(set(list(pos_lexicon_2020[pos_lexicon_2020['cosine_similarity']>=0.65]['news_word'])))

c

['개선',
 '최선',
 '장점',
 '밑돈',
 '쎄미켐동진쎄미켐',
 '재택명령',
 '감소',
 '약세',
 '오벤터스',
 '경의',
 '웃돌',
 '적극',
 '불황',
 '셰도보팅',
 '증가',
 '충족',
 '추진',
 '실패',
 '단점',
 '것이므']